<a href="https://colab.research.google.com/github/me1nna/fake-images-detection/blob/main/KNN_logreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Простая классификация на эмбеддингах CLIP

Установка зависимостей

In [ ]:
import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
import random
import os

import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision.transforms import v2
from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = "42"

In [ ]:
import PIL
print(PIL.__version__)

9.4.0


In [ ]:
!gdown --id 13rZ1HNOWsN-cTYgYHu0S3UYUHI0KAUBV
!gdown --id 1qjK-uwy20OAWCO1l39RNvjRgjsZiFqm0
!gdown --id 1Heeu9E9vI3MtkYE-_iZdGGcytm5Mw8ha
!gdown --id 1vnG3dlMb9wxo0P-noc84LUBaZNjwumMq
!gdown --id 1BW2ARIM2r0EgL5Z4A8E0O4PXB2BNRhmA

!unzip "images.zip"

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
data = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [ ]:
data

,id,target
0,gL5fWrgNPU.jpg,0
1,KKWXJDN8uM.png,1
2,Wb2I0CXlAY.jpg,0
3,G7PLhKpaf7.jpeg,1
4,DEUkeefz6Z.jpg,0
...,...,...
1007,fqoVycdxLV,1
1008,TN2ZOiNI5e.jpg,0
1009,2WMWt6CAWQ.jpg,0
1010,fQCY5n2vcF.jpg,0


In [ ]:
train_labels = data["target"]

In [ ]:
def load_images(image_paths):
    images = []
    for image_path in image_paths:
        full_path = os.path.join('/content/images', image_path)
        if not os.path.isfile(full_path):
            full_path += ".png"
        try:
            images.append(full_path)
        except Exception as e:
            print(f"Не удалось загрузить изображение: {full_path}")
            print(f"Ошибка: {e}")
            continue
    return images

In [ ]:
train_files = load_images(data["id"])
test_files = load_images(data_test["id"])

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import clip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

model, preprocess = clip.load('ViT-B/32', device='cuda')

def get_image_embeddings(image_paths):
    embeddings = []
    for path in image_paths:
        image = preprocess(Image.open(path)).unsqueeze(0).to('cuda')
        with torch.no_grad():
            embedding = model.encode_image(image).cpu().numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)

train_embeddings = get_image_embeddings(train_files)
test_embeddings = get_image_embeddings(test_files)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, train_labels)



100%|███████████████████████████████████████| 338M/338M [00:24<00:00, 14.2MiB/s]


LogisticRegression(max_iter=1000)

In [ ]:
probs = classifier.predict_proba(test_embeddings)[:, 1]
my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.000004
1,j4Rhioq7R3.jpeg,0.210262
2,rD0hgFHJUZ.jpeg,0.013969
3,aY5z1EJsJ6.jpeg,0.999998
4,qZ3IoxD2TE.jpeg,0.001743


In [ ]:
my_submit.to_csv("sub1.csv", index=False)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=3)

In [ ]:
classifier.fit(train_embeddings, train_labels)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
probs = classifier.predict_proba(test_embeddings)[:, 1]
my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.000000
1,j4Rhioq7R3.jpeg,0.000000
2,rD0hgFHJUZ.jpeg,0.333333
3,aY5z1EJsJ6.jpeg,1.000000
4,qZ3IoxD2TE.jpeg,0.333333


In [ ]:
my_submit.to_csv("sub2.csv", index=False)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(train_embeddings, train_labels)

probs = classifier.predict_proba(test_embeddings)[:, 1]
my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.0
1,j4Rhioq7R3.jpeg,0.4
2,rD0hgFHJUZ.jpeg,0.4
3,aY5z1EJsJ6.jpeg,1.0
4,qZ3IoxD2TE.jpeg,0.4


In [ ]:
my_submit.to_csv("sub3.csv", index=False)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(train_embeddings, train_labels)

probs = classifier.predict_proba(test_embeddings)[:, 1]
my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.0
1,j4Rhioq7R3.jpeg,0.0
2,rD0hgFHJUZ.jpeg,0.0
3,aY5z1EJsJ6.jpeg,1.0
4,qZ3IoxD2TE.jpeg,1.0


In [ ]:
my_submit.to_csv("sub4.csv", index=False)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=9)
classifier.fit(train_embeddings, train_labels)

probs = classifier.predict_proba(test_embeddings)[:, 1]
my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.000000
1,j4Rhioq7R3.jpeg,0.444444
2,rD0hgFHJUZ.jpeg,0.444444
3,aY5z1EJsJ6.jpeg,1.000000
4,qZ3IoxD2TE.jpeg,0.444444


In [ ]:
classifier = LogisticRegression(max_iter=1200, penalty='l2', C=1)
classifier.fit(train_embeddings, train_labels)

LogisticRegression(C=1, max_iter=1200)

In [ ]:
y_pred = classifier.predict(train_embeddings)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(train_labels, y_pred)

print(f"Точность модели на трейне: {accuracy}")


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

f1 = f1_score(train_labels, y_pred)

precision = precision_score(train_labels, y_pred)

recall = recall_score(train_labels, y_pred)

print(f"F1-score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


F1-score: 1.0
Precision: 1.0
Recall: 1.0


In [ ]:
probs = classifier.predict_proba(test_embeddings)[:, 1]
my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.000004
1,j4Rhioq7R3.jpeg,0.210262
2,rD0hgFHJUZ.jpeg,0.013969
3,aY5z1EJsJ6.jpeg,0.999998
4,qZ3IoxD2TE.jpeg,0.001743


In [ ]:
my_submit.to_csv("sub5.csv", index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7, 10, 13, 15, 17, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5)

# обучим модель KNN с поиском по сетке
grid_search.fit(train_embeddings, train_labels)

# выведем лучшие параметры
best_params = grid_search.best_params_

# применяем лучшую модель к тестовым данным и выводим score
best_knn = grid_search.best_estimator_
knn_score = best_knn.score(train_embeddings, train_labels)

print(f"Лучшие параметры KNN: {best_params}")
print(f"Score лучшей KNN модели на трейне: {knn_score}")

In [ ]:
y_pred = best_knn.predict(train_embeddings)
f1 = f1_score(train_labels, y_pred)

precision = precision_score(train_labels, y_pred)

recall = recall_score(train_labels, y_pred)

print(f"F1-score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

F1-score: 0.9479768786127167
Precision: 0.9452449567723343
Recall: 0.9507246376811594


In [ ]:
accuracy = accuracy_score(train_labels, y_pred)

print(f"Точность модели на трейне: {accuracy}")